In [1]:
import pandas as pd

# Load data GLD
df = pd.read_csv("../data/gld_clean.csv", skiprows=2)
df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df = df[['Open', 'High', 'Low', 'Close', 'Volume']]

# === Buat fitur tambahan ===
df['Return'] = df['Close'].pct_change() * 100
df['Volatility'] = df['Return'].rolling(10).std()
df['MA5'] = df['Close'].rolling(window=5).mean()
df['MA10'] = df['Close'].rolling(window=10).mean()
df['MA20'] = df['Close'].rolling(window=20).mean()

# RSI
delta = df['Close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
avg_gain = gain.rolling(14).mean()
avg_loss = loss.rolling(14).mean()
rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))

# MACD
ema_12 = df['Close'].ewm(span=12, adjust=False).mean()
ema_26 = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = ema_12 - ema_26
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

# === Buat Target ===
# Untuk regresi: harga Close hari besok
df['Target_Close'] = df['Close'].shift(-1)

# Untuk klasifikasi: apakah besok harga naik?
df['Target_UpDown'] = (df['Target_Close'] > df['Close']).astype(int)

# Drop baris NaN
df.dropna(inplace=True)

# Simpan ke file baru
df.to_csv("../data/gld_features.csv")

# Cek hasil
print(df.tail())


                  Open        High         Low       Close   Volume    Return  \
Date                                                                            
2024-12-23  241.600006  241.669998  240.649994  240.960007  5835500 -0.470880   
2024-12-24  241.490005  241.660004  240.820007  241.440002  2421000  0.199201   
2024-12-26  242.389999  243.559998  242.199997  243.070007  4645100  0.675118   
2024-12-27  241.199997  241.949997  241.050003  241.399994  4728100 -0.687050   
2024-12-30  241.080002  241.080002  239.580002  240.630005  3522500 -0.318968   

            Volatility         MA5        MA10        MA20        RSI  \
Date                                                                    
2024-12-23    1.113767  241.172003  244.186002  243.979000  44.601999   
2024-12-24    0.997480  240.672003  243.471002  243.927000  44.073412   
2024-12-26    0.962296  241.434006  242.682002  243.933001  50.387895   
2024-12-27    0.887302  241.794003  242.094002  243.828500  47.2949